In [1]:
!pip install datasets
!pip install pandas
!pip install nltk
!pip install contractions
!pip install inflect
!pip install pandas
!pip install scikit-learn
!pip install scikit-multilearn
!pip install tqdm

In [2]:
from pathlib import Path

import datasets
import pandas as pd

import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import sys
from random import randbytes

In [3]:
dataset = datasets.load_from_disk(str("./prepared-dataset"))

In [4]:
!pip install transformers
!pip install accelerate

In [5]:
import pandas as pd
import warnings
import numpy as np
import pandas
from datasets import Dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, classification_report, f1_score
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score, precision_score, recall_score

GSCV_parameters = [{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
                   {'penalty': ['none', 'elasticnet', 'l1', 'l2']},
                   {'C': [0.001, 0.01, 0.1, 1, 10, 100]}]

warnings.filterwarnings("ignore")

def classifier(dataset, model):
    # Use real X and y from dataset
    print(dataset)
    dataset = dataset.to_pandas()
    X = dataset["message"]
    y = dataset["is_toxic"]
    scores = []
    matrices = []

    x_train, x_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=46, shuffle=True)
    if model == "classic_ml":
        tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), lowercase=True, max_features=150000)
        train_tfidf = tfidf_vec.fit_transform(x_train)
        test_tfidf = tfidf_vec.transform(x_val)
        print(tfidf_vec.get_feature_names_out()[:10])

        res_model = model_classic_train(train_tfidf, y_train)
        eval_res = eval_fun_br(test_tfidf, y_val, res_model)
        print({'tn': eval_res[1][0, 0], 'fp': eval_res[1][0, 1],
               'fn': eval_res[1][1, 0], 'tp': eval_res[1][1, 1]})


    elif model in ["roberta-base", "microsoft/codebert-base"]:
        x_tr, x_test, y_tr, y_test = train_test_split(
            x_train,
            y_train,
            test_size=0.8,
            random_state=46,
            shuffle=True)
        tokenizer = RobertaTokenizer.from_pretrained(model)
        tf_bert = lambda examples: tokenizer(examples["text"], truncation=True, padding="max_length")

        trd = Dataset.from_pandas(pd.DataFrame({"text": x_tr, "labels": y_tr})).map(tf_bert, batched=True)
        tsd = Dataset.from_pandas(pd.DataFrame({"text": x_test, "labels": y_test})).map(tf_bert, batched=True)
        vald = Dataset.from_pandas(pd.DataFrame({"text": x_val, "labels": y_val})).map(tf_bert, batched=True)
        res_model = bert_model(dataset, trd, tsd, model)
        metrics = res_model.evaluate(vald)
        print(f"MODEL \"{model}\" VALIDATION RESULTS")
        print("@@@@@@@@@@@@@@@@@@@@@@@@")
        print(f"Accuracy: {metrics['eval_accuracy']:.3f}")
        print(f"Precision: {metrics['eval_precision']:.3f}")
        print(f"Recall: {metrics['eval_recall']:.3f}")
        print(f"F1 Score: {metrics['eval_f1']:.3f}")
        print("--------------------------")

    elif model == "all":
        tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), lowercase=True, max_features=150000)
        train_tfidf = tfidf_vec.fit_transform(x_train)
        test_tfidf = tfidf_vec.transform(x_val)
        print(tfidf_vec.get_feature_names_out()[:10])
        res_model = model_classic_train(train_tfidf, y_train)

        x_tr, x_test, y_tr, y_test = train_test_split(
            x_train,
            y_train,
            test_size=0.8,
            random_state=46,
            shuffle=True)

        tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
        tf_bert = lambda examples: tokenizer(examples["text"], truncation=True, padding="max_length")

        trd = Dataset.from_pandas(pd.DataFrame({"text": x_tr, "labels": y_tr})).map(tf_bert, batched=True)
        tsd = Dataset.from_pandas(pd.DataFrame({"text": x_test, "labels": y_test})).map(tf_bert, batched=True)
        vald_A = Dataset.from_pandas(pd.DataFrame({"text": x_val, "labels": y_val})).map(tf_bert, batched=True)

        roberta = bert_model(dataset, trd, tsd, "roberta-base")

        tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
        tf_bert = lambda examples: tokenizer(examples["text"], truncation=True, padding="max_length")

        trd = Dataset.from_pandas(pd.DataFrame({"text": x_tr, "labels": y_tr})).map(tf_bert, batched=True)
        tsd = Dataset.from_pandas(pd.DataFrame({"text": x_test, "labels": y_test})).map(tf_bert, batched=True)
        vald_B = Dataset.from_pandas(pd.DataFrame({"text": x_val, "labels": y_val})).map(tf_bert, batched=True)

        codebert = bert_model(dataset, trd, tsd, "microsoft/codebert-base")
        print("MODELS FULL COMPARE")
        print("--------------------")
        print("Best Logistic Regression")
        print(eval_fun_br(test_tfidf, y_val, res_model))

        metrics = roberta.evaluate(vald_A)
        print(f"MODEL \"ROBERTA\" RESULTS")
        print("@@@@@@@@@@@@@@@@@@@@@@@@")
        print(f"Accuracy: {metrics['eval_accuracy']:.3f}")
        print(f"Precision: {metrics['eval_precision']:.3f}")
        print(f"Recall: {metrics['eval_recall']:.3f}")
        print(f"F1 Score: {metrics['eval_f1']:.3f}")
        print("--------------------------")

        metrics = codebert.evaluate(vald_B)
        print(f"MODEL \"CODEBERT\" RESULTS")
        print("@@@@@@@@@@@@@@@@@@@@@@@@")
        print(f"Accuracy: {metrics['eval_accuracy']:.3f}")
        print(f"Precision: {metrics['eval_precision']:.3f}")
        print(f"Recall: {metrics['eval_recall']:.3f}")
        print(f"F1 Score: {metrics['eval_f1']:.3f}")
        print("--------------------------")

    else:
        raise Exception("Invalid model name")


def model_classic_train(x_train, y_train):
    lr_clf = LogisticRegression()
    scoring = {"Accuracy": "accuracy", "F1_Score": "f1",
               "AUC": "roc_auc",
               "NegLogLoss": "neg_log_loss",
               "Precision": "precision",
               "Recall": "recall"
               }
    reg_cv = GridSearchCV(lr_clf, GSCV_parameters, cv=10, scoring=scoring, refit="F1_Score", verbose=0)
    reg_cv.fit(x_train, y_train)  # grid search CV makes division of dataset on test&train by itself

    print("GridSearch mean results beetwere:")
    print("CROSS VALID FINISHED, RESULTS:")
    print("Best params: ", reg_cv.best_estimator_.get_params())

    print("Best F1 Score result: ", reg_cv.best_score_)
    print("Index of the best model is: ", np.argmax(np.nan_to_num(reg_cv.cv_results_["mean_test_F1_Score"])))
    print("Mean within K10Folds validations F1 Score result: ",
          np.max(np.nan_to_num(reg_cv.cv_results_["mean_test_F1_Score"])))
    print("Mean within K10Folds validations Accuracy result: ",
          np.max(np.nan_to_num(reg_cv.cv_results_["mean_test_Accuracy"])))
    print("Mean within K10Folds validations AUC result: ", np.max(np.nan_to_num(reg_cv.cv_results_["mean_test_AUC"])))
    print("Mean within K10Folds validations NegLogLoss result: ",
          np.max(np.nan_to_num(reg_cv.cv_results_["mean_test_NegLogLoss"])))
    print("Mean within K10Folds validations Precision result: ",
          np.max(np.nan_to_num(reg_cv.cv_results_["mean_test_Precision"])))
    print("Mean within K10Folds validations Recall result: ",
          np.max(np.nan_to_num(reg_cv.cv_results_["mean_test_Recall"])))
    return reg_cv


def eval_fun_br(x_test, y_test, classifier):
    y_labels = np.array(classifier.predict(x_test))
    conf_matrix = confusion_matrix(y_test.to_numpy(), y_labels)
    f1 = f1_score(y_test, y_labels)
    precision = precision_score(y_test, y_labels)
    recall = recall_score(y_test, y_labels
                          )
    print("PREC: ", precision)
    print("REC: ", recall)
    print("F1 Score: ", f1)

    return (pandas.DataFrame({"Accuracy": np.round(np.mean([accuracy_score(y_test.to_numpy(),
                                                                           y_labels) for i in range(6)]), 3),
                              "F1": np.round(f1, 3),
                              "Precision": np.round(precision, 3),
                              "Recall": np.round(recall, 3),
                              "AUC": np.round(np.mean([roc_auc_score(y_test.to_numpy(),
                                                                     y_labels) for i in range(6)]), 3),
                              "Log loss": np.round(np.mean([log_loss(y_test.to_numpy(),
                                                                     y_labels) for i in range(6)]), 3)}, index=[0]),
            conf_matrix)


def bert_compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="weighted"
    )

    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


def bert_model(dataset, trd, tsd, model):
    model_impl = RobertaForSequenceClassification.from_pretrained(model, num_labels=2)

    targs = TrainingArguments(
        output_dir=f"./trained-{model}",
        eval_strategy="epoch",
        learning_rate=4e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=1,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=10,
    )

    trainer = Trainer(
        model=model_impl,
        args=targs,
        train_dataset=trd,
        eval_dataset=tsd,
        compute_metrics=bert_compute_metrics,
    )

    trainer.train()
    metrics = trainer.evaluate()

    print(f"MODEL \"{model}\" RESULTS")
    print("@@@@@@@@@@@@@@@@@@@@@@@@")
    print(f"Accuracy: {metrics['eval_accuracy']:.3f}")
    print(f"Precision: {metrics['eval_precision']:.3f}")
    print(f"Recall: {metrics['eval_recall']:.3f}")
    print("--------------------------")
    print(f"F1 Score: {metrics['eval_f1']:.3f}")

    return trainer


In [6]:
classifier(dataset, "all")

Dataset({
    features: ['message', 'is_toxic', '__index_level_0__'],
    num_rows: 12494
})
['aa' 'aa ae' 'aa cc' 'aa reserve' 'aa review' 'aa service' 'aaargh'
 'aaargh damn' 'aae' 'aae cfcbc']
GridSearch mean results beetwere:
CROSS VALID FINISHED, RESULTS:
Best params:  {'C': 100, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Best F1 Score result:  0.6825115351022284
Index of the best model is:  14
Mean within K10Folds validations F1 Score result:  0.6825115351022284
Mean within K10Folds validations Accuracy result:  0.9013514514514513
Mean within K10Folds validations AUC result:  0.9315673577968143
Mean within K10Folds validations NegLogLoss result:  0.0
Mean within K10Folds validations Precision result:  0.9390554232153455
Mean within K10Folds validations Recall r

Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Map:   0%|          | 0/7996 [00:00<?, ? examples/s]

Map:   0%|          | 0/2499 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Map:   0%|          | 0/7996 [00:00<?, ? examples/s]

Map:   0%|          | 0/2499 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.251000,0.329692,0.872311,0.879345,0.895773,0.872311


MODEL "microsoft/codebert-base" RESULTS
@@@@@@@@@@@@@@@@@@@@@@@@
Accuracy: 0.872
Precision: 0.896
Recall: 0.872
--------------------------
F1 Score: 0.879
MODELS FULL COMPARE
--------------------
Best Logistic Regression
PREC:  0.8303964757709251
REC:  0.7495029821073559
F1 Score:  0.7878787878787878
(   Accuracy     F1  Precision  Recall    AUC  Log loss
0     0.919  0.788       0.83    0.75  0.855     2.928, array([[1919,   77],
       [ 126,  377]], dtype=int64))
MODEL "ROBERTA" RESULTS
@@@@@@@@@@@@@@@@@@@@@@@@
Accuracy: 0.868
Precision: 0.881
Recall: 0.868
F1 Score: 0.873
--------------------------
MODEL "CODEBERT" RESULTS
@@@@@@@@@@@@@@@@@@@@@@@@
Accuracy: 0.871
Precision: 0.890
Recall: 0.871
F1 Score: 0.877
--------------------------


In [7]:
classifier(dataset, "classic_ml")

Dataset({
    features: ['message', 'is_toxic', '__index_level_0__'],
    num_rows: 12494
})
['aa' 'aa ae' 'aa cc' 'aa reserve' 'aa review' 'aa service' 'aaargh'
 'aaargh damn' 'aae' 'aae cfcbc']
GridSearch mean results beetwere:
CROSS VALID FINISHED, RESULTS:
Best params:  {'C': 100, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Best F1 Score result:  0.6825115351022284
Index of the best model is:  14
Mean within K10Folds validations F1 Score result:  0.6825115351022284
Mean within K10Folds validations Accuracy result:  0.9013514514514513
Mean within K10Folds validations AUC result:  0.9315673577968143
Mean within K10Folds validations NegLogLoss result:  0.0
Mean within K10Folds validations Precision result:  0.9390554232153455
Mean within K10Folds validations Recall r

In [8]:
classifier(dataset, "roberta-base")

Dataset({
    features: ['message', 'is_toxic', '__index_level_0__'],
    num_rows: 12494
})


Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Map:   0%|          | 0/7996 [00:00<?, ? examples/s]

Map:   0%|          | 0/2499 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.193900,0.322474,0.863182,0.871600,0.892157,0.863182


MODEL "roberta-base" RESULTS
@@@@@@@@@@@@@@@@@@@@@@@@
Accuracy: 0.863
Precision: 0.892
Recall: 0.863
--------------------------
F1 Score: 0.872
MODEL "roberta-base" VALIDATION RESULTS
@@@@@@@@@@@@@@@@@@@@@@@@
Accuracy: 0.864
Precision: 0.890
Recall: 0.864
F1 Score: 0.871
--------------------------


In [9]:
classifier(dataset, "microsoft/codebert-base")

Dataset({
    features: ['message', 'is_toxic', '__index_level_0__'],
    num_rows: 12494
})


Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Map:   0%|          | 0/7996 [00:00<?, ? examples/s]

Map:   0%|          | 0/2499 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.357600,0.328968,0.869560,0.875886,0.889076,0.869560


MODEL "microsoft/codebert-base" RESULTS
@@@@@@@@@@@@@@@@@@@@@@@@
Accuracy: 0.870
Precision: 0.889
Recall: 0.870
--------------------------
F1 Score: 0.876
MODEL "microsoft/codebert-base" VALIDATION RESULTS
@@@@@@@@@@@@@@@@@@@@@@@@
Accuracy: 0.863
Precision: 0.876
Recall: 0.863
F1 Score: 0.868
--------------------------
